In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [22]:
df_pitching = pd.read_csv('../input/pitching/Pitching.csv')
pitchers = df_pitching[['playerID','yearID']]

In [23]:
df_allstar = pd.read_csv('../input/baseball/AllstarFull.csv')
df_allstar['GP'] = df_allstar['GP'].replace(0,1)
df_allstar.head()

In [24]:
df_name = pd.read_csv('../input/baseball/People.csv')
df_name['name'] = df_name['nameFirst']+' '+df_name['nameLast']
df_name = df_name.drop(['birthYear','birthMonth','birthDay','birthCountry','birthState','birthCity','deathYear','deathMonth','deathDay','deathCountry','deathState','deathCity','nameFirst','nameLast','nameGiven','weight','height','bats','throws','debut','finalGame','retroID','bbrefID'],axis=1)
df_name

In [25]:
df_batting = pd.read_csv('../input/baseball/Batting.csv')
df_batting = df_batting[df_batting['yearID'] >= 1933]
df_batting['PA'] = df_batting['AB'] + df_batting['BB'] + df_batting['HBP'] + df_batting['SH'] + df_batting['SF']
df_batting['OBP'] = (df_batting['H']+ df_batting['BB'] + df_batting['HBP'])/df_batting['PA']
df_batting['SLG'] = ((df_batting['H']- df_batting['2B']- df_batting['3B']- df_batting['HR']) + 2*df_batting['2B'] + 3*df_batting['3B'] + 4*df_batting['HR'])/df_batting['AB']
df_batting['OPS'] = df_batting['OBP'] + df_batting['SLG']
df_batting = df_batting.drop(columns=['IBB', 'SF', 'CS','GIDP', 'SH'])
print(len(df_batting))
df_batting

In [26]:
df_batting = df_batting.groupby(['playerID','yearID']).sum()
df_batting.head(100)

In [27]:
df1 = df_batting.merge(df_allstar, how='left', on=['playerID','yearID'])
df1['GP'] = df1['GP'].fillna(0)
df1 = df1.drop('gameNum', axis=1)
df1 = df1.drop('gameID', axis=1)
df1 = df1.drop('teamID', axis=1)
df1 = df1.drop('lgID', axis=1)
df1 = df1.drop('startingPos', axis=1)
df1 = df1.drop('stint', axis=1)
df1 = df1.sort_values(by=['yearID', 'playerID'])

np.random.seed(20)
df2 = df1.sample(frac=.75)
df2 = df2[df2.GP != 1]
len(df2)
df_new = df1.merge(df2, how='left', indicator=True)
df_new = df_new[df_new['_merge'] == 'left_only']
df1=df_new
df1=df1.drop(['_merge'], axis=1)
df1

In [28]:
print(len(df1))
d=df1['playerID'].isin(pitchers.playerID.values)
"""
print(len(df1))
df1.loc[~df1['playerID'].isin(pitchers.playerID.values)]
len(df1)
df1.loc[df1['playerID'] == 'ohtansh01']
'ohtansh01'.isin(pitchers.playerID.values)
"""
df1=df1[~d]
len(df1)

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
#from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [30]:
df = df1.values
names = df[:,0]
df = np.delete(df, 0, 1)
Y = df[:,17].astype(int)
X = df[:,0:17].astype(float)
print(sum(Y))

In [31]:
def create_larger():
    # create model
    model = Sequential()
    model.add(Dense(17, input_shape=(17,), activation='relu'))
    #model.add(Dense(16, activation='relu'))
    #model.add(Dense(15, activation='relu'))
    model.add(Dense(14, activation='relu'))
    #model.add(Dense(13, activation='relu'))
    #model.add(Dense(12, activation='relu'))
    model.add(Dense(11, activation='relu'))
    #model.add(Dense(10, activation='relu'))
    #model.add(Dense(9, activation='relu'))
    model.add(Dense(8, activation='relu'))
    #model.add(Dense(7, activation='relu'))
    #model.add(Dense(6, activation='relu'))
    model.add(Dense(5, activation='relu'))
    #model.add(Dense(4, activation='relu'))
    model.add(Dense(3, activation='relu'))
    #model.add(Dense(2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','FalseNegatives', 'FalsePositives'])
    return model

In [33]:
model = create_larger()
model.fit(X,Y, epochs=100, verbose=1)

In [34]:
df2 = df1[df1['yearID']==2021]
#df2 = df2[df2['GP']==1]
np_df2 = df2.values
np_df2 = np.delete(np_df2, 0, 1)
Y = np_df2[:,17].astype(int)
X = np_df2[:,0:17].astype(float)

In [35]:
def sigmoid(data):
    for x in range(len(data)):
        if data[x][0]>=0.5:
            data[x][0]=1
        else:
            data[x][0]=0
    return data
d = model.predict_on_batch(X)
d=sigmoid(d)

In [36]:
df2['res'] = d
df2[df2.res==1]

In [37]:
df2[df2.GP==1]

In [39]:
df2.to_csv('results.csv')